In [25]:
from six.moves import cPickle as pickle
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint

Loading the data

In [26]:
pfile = r"./data/ABCD_Datasets.pickle"
with (open(pfile, "rb")) as openfile:
    while True:
        try:
            ABCD_Datasets = pickle.load(openfile)
        except EOFError:
            break

X_train_D = ABCD_Datasets["train_datasets"]["D"]
Y_train_D = ABCD_Datasets["train_labels"]["D"]
X_test_D = ABCD_Datasets["test_datasets"]["D"]
Y_test_D = ABCD_Datasets["test_labels"]["D"]


one hot-encoding and creating one-hot encoding, quick check

In [27]:
# one-hot encode the labels
num_classes = 10
Y_train_D_hot = keras.utils.to_categorical(Y_train_D-1, num_classes)
Y_test_D_hot = keras.utils.to_categorical(Y_test_D-1, num_classes)

# break training set into training and validation sets
(X_train, X_valid) = X_train_D[2000:], X_train_D[:2000]
(Y_train, Y_valid) = Y_train_D_hot[2000:], Y_train_D_hot[:2000]
X_test = X_test_D
Y_test = Y_test_D_hot

# print shape of training set
print('x_train shape:', X_train.shape)

# print number of training, validation, and test images
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'validation samples')


x_train shape: (17800, 2048, 2)
17800 train samples
750 test samples
2000 validation samples


Let's start with simple methods.
1. Rondom guess
2. logistic regression (or 2 layers NN model)

In [33]:
# only using 1 channel now
CNNch = 1

# epch
ne = 10

# define the model
modelLog = Sequential()
modelLog.add(Flatten(input_shape = (2048,CNNch)))
modelLog.add(Dense(10, activation='softmax'))

modelLog.summary()
# total param 2,566,642
# ref  CNN       53,830
modelLog.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

hist = modelLog.fit(X_train[:,:,0:CNNch], Y_train, batch_size=32, epochs=ne,
          validation_data=(X_valid[:,:,0:CNNch], Y_valid), 
          verbose=1, shuffle=True)

score = modelLog.evaluate(X_test[:,:,0:CNNch], Y_test, verbose=0)
print('\n', 'Logistic Regression Test accuracy:', score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_21 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 10)                20490     
Total params: 20,490
Trainable params: 20,490
Non-trainable params: 0
_________________________________________________________________
Train on 17800 samples, validate on 2000 samples
Epoch 1/10
17800/17800 [==============================] - 5s 288us/step - loss: 2.3623 - acc: 0.2063 - val_loss: 2.3487 - val_acc: 0.2000
Epoch 2/10
17800/17800 [==============================] - 4s 243us/step - loss: 2.2467 - acc: 0.2658 - val_loss: 2.3422 - val_acc: 0.2220
Epoch 3/10
17800/17800 [==============================] - 4s 217us/step - loss: 2.1817 - acc: 0.3004 - val_loss: 2.3353 - val_acc: 0.2440
Epoch 4/10
17800/17800 [==============================] - 4s 241us/step

1D-CNN

From the moddel suggested in the paper 
* the 2nd layer # of filter(kernel) is reduced from 32 to 16.
* the 3rd layer # of filter(kernel) is reduced from 64 to 32.
* the 4th layer # of filter(kernel) is reduced from 64 to 32.
* the 5th layer # of filter(kernel) is reduced from 64 to 32.
* the last layer fullly connected layer is reduced from 100 to 50.

Just to make the number of variable even smaller than the logistic regression above.


In [53]:
CNNch = 2

# epch
ne = 10

modelC2 = Sequential()
#1
modelC2.add(Conv1D(filters=16, kernel_size=64,strides = 16, padding='same', activation='relu', 
                        input_shape=(2048, CNNch)))
modelC2.add(MaxPooling1D(pool_size=2))
#2
modelC2.add(Conv1D(filters=16, kernel_size=3, strides = 1, padding='same', activation='relu'))
modelC2.add(MaxPooling1D(pool_size=2))
#3
modelC2.add(Conv1D(filters=32, kernel_size=3, strides = 1, padding='same', activation='relu'))
modelC2.add(MaxPooling1D(pool_size=2))
modelC2.add(Dropout(0.2))
#4
modelC2.add(Conv1D(filters=32, kernel_size=3, strides = 1, padding='same', activation='relu'))
modelC2.add(MaxPooling1D(pool_size=2))
modelC2.add(Dropout(0.2))
#5
modelC2.add(Conv1D(filters=32, kernel_size=3, strides = 1, padding='same', activation='relu'))
#paper no padding?, Yes, to make 5th layer output 6 width and 3 after pooling
#-> same seems to perform little better because of more parameter? 
# little diffrernt from the paper but keep it as padding = 'same'
modelC2.add(MaxPooling1D(pool_size=2))  

modelC2.add(Flatten())
modelC2.add(Dense(50, activation='relu'))
modelC2.add(Dropout(0.2))
modelC2.add(Dense(10, activation='softmax'))

modelC2.summary()


# compile the model
modelC2.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

# train the model
checkpointer = ModelCheckpoint(filepath='CNNC2.weights.best.hdf5', verbose=1, 
                               save_best_only=True)

hist = modelC2.fit(X_train[:,:,0:CNNch], Y_train, batch_size=32, epochs=ne,
          validation_data=(X_valid[:,:,0:CNNch], Y_valid), callbacks=[checkpointer], 
          verbose=1, shuffle=True)

# load the weights that yielded the best validation accuracy
modelC2.load_weights('CNNC2.weights.best.hdf5')

# evaluate and print test accuracy
score = modelC2.evaluate(X_test[:,:,0:CNNch], Y_test, verbose=0)
print('\n', 'CNN Test accuracy:', score[1])

score = modelC2.evaluate(X_train[:,:,0:CNNch], Y_train, verbose=0)
print('\n', 'CNN train accuracy:', score[1])

score = modelC2.evaluate(X_valid[:,:,0:CNNch], Y_valid, verbose=0)
print('\n', 'CNN validation accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_111 (Conv1D)          (None, 128, 16)           2064      
_________________________________________________________________
max_pooling1d_111 (MaxPoolin (None, 64, 16)            0         
_________________________________________________________________
conv1d_112 (Conv1D)          (None, 64, 16)            784       
_________________________________________________________________
max_pooling1d_112 (MaxPoolin (None, 32, 16)            0         
_________________________________________________________________
conv1d_113 (Conv1D)          (None, 32, 32)            1568      
_________________________________________________________________
max_pooling1d_113 (MaxPoolin (None, 16, 32)            0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 16, 32)            0         
__________